In [ ]:
import xarray as xr
import numpy as np
import math
import datetime
import yaml

import matplotlib.pyplot as plt
import cmocean as cm  

In [ ]:
#This file contains configuration details like API keys and passwords
global_vars = yaml.safe_load(open('../config.yml', 'r') )

In [ ]:
#This has custom functions - namely finding timeframes and graphing
%run ./00_custom_functions.ipynb   

In [ ]:
#set data folders for input and output
cloud = False
if cloud:
    input_folder = global_vars['reconstruction_folder_cloud']
    reconstructed_file = input_folder + 'pCO2_LEAP_XGBoost-Reconstructed_202001-202212.nc'
    #seaflux_file = '/data/artemis/observations/SeaFlux/SeaFlux_v2022.01_no_pco2.nc'
    seaflux_file = global_vars['download_folder_cloud'] + '\SeaFlux\originals\SeaFlux_LDEO_SeaFlux-v202301-all_1982-2022.nc'
    coastal_filling_file = '/data/artemis/observations/SeaFlux/Coastal_fill_scaling_LDEOtmp.nc'
else:
    input_folder = global_vars['reconstruction_folder_local']
    reconstructed_file = input_folder + 'pCO2_LEAP_XGBoost-fco2-residual-reconstructed_198201-202212.nc'
    seaflux_file = global_vars['download_folder_local'] + '\SeaFlux\originals\SeaFlux_LDEO_SeaFlux-v202301-all_1982-2022.nc'
    coastal_filling_file = r'C:\Users\Devan\Downloads\datasets\LEAP_CO2\artemis_copy\Coastal_fill_scaling_LDEOtmp.nc'

output_folder = input_folder   #Because we are inputting and outputting processed files, the folder can be the same

flux_variable = 'fco2'  #for differentiating between pco2 flux and fco2 flux. Used for naming as well

#### Ensure Common Gridding

In [ ]:
recon_ens = xr.load_dataset(reconstructed_file)
seaflux = xr.open_dataset(seaflux_file)
coast_new = xr.open_dataset(coastal_filling_file)

In [ ]:
#This chunk is a TEMPORARY update to the coastal filling file to copy 2021 data over to 2022 so we can view 2022 flux
# ideally we would update coastal file with 2022 using normal methodology - TODO
import pandas as pd
coast_tmp = np.empty([480+12,180,360])
ttime = pd.date_range(start='1982-01', end='2022-12',freq='MS') + np.timedelta64(14, 'D')
coast_tmp[0:480,0:180,0:360] = coast_new.coast_fill
coast_tmp[480:,0:180,0:360] = coast_new.coast_fill.sel(time=slice('2021','2021')) #fill 2022 with values from 2021
coast_tmp = xr.Dataset({'coast_fill':(["time","lat","lon"],coast_tmp)},
                 coords={'time':(['time'],ttime),'lat':(['lat'],coast_new.lat.values),'lon':(['lon'],coast_new.lon.values) })
coast_new = coast_tmp

In [ ]:
# To match the reconstructed data, we need each file to be in this format: time, ylat, xlon 
# with time ranges on the 15th of the month, 
# and ylat and xlon on 1 degree resolution (ylat from -89.5 to 89.5; xlon from -179.5 to 179.5)
# This code checks these conditions
#If anything errors out, we would need to reformat the input data prior to this script

if True: #these are known changes required from the 2021 version
    seaflux = seaflux.rename({'lat': 'ylat', 'lon':'xlon'})
    seaflux = seaflux.transpose('time','ylat','xlon',...)
    coast_new = coast_new.rename({'lat': 'ylat', 'lon':'xlon'})
    coast_new = coast_new.transpose('ylat','xlon',...)

#check coordinate naming
assert set(list(seaflux.coords)) >= {'time','ylat','xlon'}
assert set(list(coast_new.coords)) >= {'time','ylat','xlon'}

#check mid-month date
assert set(list([x.astype('datetime64[s]').item().day for x in seaflux.time.data])) == {15}
assert set(list([x.astype('datetime64[s]').item().day for x in coast_new.time.data])) == {15}

#check grid resolution
assert set(list(seaflux.ylat.values)) <= set([x+.5 for x in range(-90, 90, 1)])
assert set(list(seaflux.xlon.values)) <= set([x+.5 for x in range(-180,180,1)])
assert set(list(coast_new.ylat.values)) <= set([x+.5 for x in range(-90, 90, 1)])
assert set(list(coast_new.xlon.values)) <= set([x+.5 for x in range(-180,180,1)])

#### Find Common Timeframe

In [ ]:
start_yearmonth, end_yearmonth = find_least_date_range([recon_ens, seaflux, coast_new]) #ocean area does not have a time component so is not included

#start_yearmonth = '2020-01'  #just for testing/debugging
#end_yearmonth = '2021-12'   #just for testing/debugging

print(f'Annual flux calculations will output from: {start_yearmonth} to {end_yearmonth}')

## Calculate Flux

In [ ]:
#filter to same time frame
seaflux_filter = seaflux.sel(time=slice(str(start_yearmonth),str(end_yearmonth))) 
coast_filter = coast_new.coast_fill.sel(time=slice(str(start_yearmonth),str(end_yearmonth))) 
recon_ens_filter = recon_ens.sel(time=slice(str(start_yearmonth),str(end_yearmonth))) 

In [ ]:
%%time
wind_prods = ["CCMP2","ERA5","JRA55"]
kw1 = seaflux_filter.kw.sel(wind=wind_prods) #just get the 3 winds we want. 
kw = kw1 * 87.6        # --> cm/hr now m/yr
kw = kw.where(kw > 0)  # solubility (mol/m3/uatm) and kw converted from (cm/hr) to (m/s)
k0 = seaflux_filter.sol/1000  # mol / m3 / uatm. Note that in new version of 
icef = seaflux_filter.ice    # ice fraction 
ice_weighting = 1 - icef.fillna(0)

In [ ]:
#Now process reconstructed ocean_co2 and create pco2 delta
ocean_co2 = recon_ens_filter[flux_variable+'_reconstructed'] #ex. recon_ens_filter.pco2_reconstructed
ocean_co2_filled = ocean_co2.fillna(coast_filter)  
delta_co2 = ocean_co2_filled - seaflux_filter.fco2atm

In [ ]:
#calculate flux
flux = ( k0 * kw * delta_co2 * ice_weighting ) # mol/m2/yr
flux_xr = xr.Dataset({'flux':(["time","ylat","xlon","wind"],flux.data)},
                        coords={'time':(['time'],flux.time.values),
                                'ylat':(['ylat'],flux.ylat.values),
                                'xlon':(['xlon'],flux.xlon.values),
                                'wind':(['wind'],flux.wind.values)  })

flux_xr.flux.attrs['description'] = flux_variable+"-Residual integrated CO2 Flux (XGBoost ensemble mean)"
flux_xr.flux.attrs['units'] = 'mol/m^2/yr (negative into ocean), no river flux adjustment done here'
flux_xr.attrs['ocean_co2_source'] = str(reconstructed_file)
flux_xr.attrs['seaflux_source'] = str(seaflux_file)
flux_xr.attrs['coast_filling_source'] = str(coastal_filling_file)
flux_xr.attrs['created'] = str(datetime.datetime.now())
#flux_xr.attrs['code'] = "/home/vbennington/pCO2_Residual/calc_fluxes.ipynb"
flux_xr

In [ ]:
#output optionally
if True: 
    output_netcdf_with_date(flux_xr, output_folder+'', 'pCO2_LEAP_'+flux_variable+'-residual-flux-monthly-3winds') 

In [ ]:
plot_data = flux_xr.flux.mean("time").mean("wind")  
levelspace = np.linspace(-5,5,11) 
fig,ax = plt.subplots(1,1,figsize=(8,3))
x0=ax.contourf(flux_xr.xlon,flux_xr.ylat,plot_data,levels=levelspace,cmap=cm.cm.balance)
plt.colorbar(x0,ax=ax).set_label('uatm');
ax.set_title("Mean "+'pCO2_flux'+ " across time/wind product");

## Enhancing / Summarizing Flux

In [ ]:
# This code adjusts for ocean acea and summarizes glboally and annually

area = seaflux_filter.area #sarea.area_ocean
flux_tmp = flux_xr.flux * area * 12     # g/yr; flux_ens is in mol/m2/yr so *area gets you mol/yr and then *12 is g/yr
#flux_tmp = flux_tmp.where(mask['open_ocean']>=1).sum(['lat','lon'])  #this is old code; not required as land is already nan
flux_all_regions = flux_tmp.sum(['ylat','xlon'])  #sum over the region to get total flux in g/yr
flux_all_regions_corrected = flux_all_regions.where(flux_all_regions!=0)  #this is to correct for anomalies in data
#flux_all_regions_corrected = flux_all_regions_corrected/1e15 #(optionally) divide by 1e15 to get in Pg/yr

In [ ]:
# Now compose an xarray to write out a file
# Note, we are only use 1 LDEO product so this code has been simplified from a previous version

annual_fluxes = flux_all_regions_corrected.groupby("time.year").mean("time")
flux_file_yearly = xr.Dataset({
                        'flux':(["time","wind"],flux_all_regions_corrected.data),  #calculated above
                        'annual_flux':(["year","wind"],annual_fluxes.data)},
                        coords={'time':(['time'],flux_all_regions_corrected.time.values),
                                'year':(['year'],range(int(start_yearmonth[0:4]), int(end_yearmonth[0:4])+1)),  #this is just a list of years
                                'wind':(['wind'],flux_all_regions_corrected.wind.values)  })

flux_file_yearly.flux.attrs['description'] = flux_variable+"-Residual integrated CO2 Flux (XGBoost ensemble mean)"
flux_file_yearly.flux.attrs['units'] = "g C /year (negative into ocean), no river flux adjustment done here"
flux_file_yearly.annual_flux.attrs['units'] = "g C /year (negative into ocean), no river flux adjustment done here"
flux_file_yearly.attrs['ocean_co2_source'] = str(reconstructed_file)
flux_file_yearly.attrs['seaflux_source'] = str(seaflux_file)
flux_file_yearly.attrs['coast_filling_source'] = str(coastal_filling_file)
flux_file_yearly.attrs['created'] = str(datetime.datetime.now())
#flux_xr.attrs['code'] = "/home/vbennington/pCO2_Residual/calc_fluxes.ipynb"

flux_file_yearly

In [ ]:
#output flux to netcdf file
output_netcdf_with_date(flux_file_yearly, output_folder+'', 'pCO2_LEAP_'+flux_variable+'-residual-flux-monthly-3winds-summary')

In [ ]:
river_carbon_adjustment = 0.49   #Pg/yr adjustment to be applied on all products 
plot_data = flux_file_yearly.flux.groupby("time.year").mean("time").mean('wind')/1e15 - river_carbon_adjustment

fig = plt.figure(figsize=(8,3))
plt.plot(flux_file_yearly.year,plot_data)
plt.grid(True)
plt.title('Annual CO2 Flux \n (avg 3 wind products; with riverine CO2 adj)'); 
plt.xlabel('Time'); plt.ylabel(flux_variable + ' Flux (Pg/yr)');